In [ ]:
# run python files
# !python /content/drive/MyDrive/Colab\ Notebooks/OCR/src/model_definition.py

In [68]:
import os
import random
import sys
from sklearn.model_selection import train_test_split
from importlib import reload

sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/OCR/src')

import utils
import model_definition
import model_train

reload(utils)
reload(model_definition)
reload(model_train)

<module 'model_train' from '/content/drive/MyDrive/Colab Notebooks/OCR/src/model_train.py'>

Get list of images

In [8]:
IMG_LIST_DIR = '/content/drive/MyDrive/Colab Notebooks/OCR/data/PageImg/'
SEG_LIST_DIR = '/content/drive/MyDrive/Colab Notebooks/OCR/data/PageSeg/'

image_list = os.listdir(IMG_LIST_DIR)
image_list = [filename.split(".")[0] for filename in image_list]

seg_list = os.listdir(SEG_LIST_DIR)
seg_list = [filename.split(".")[0] for filename in seg_list]

Split data into train and test

In [9]:
X_train, X_test, y_train, y_test = train_test_split(image_list, seg_list, test_size = 0.3, shuffle = True)

Import model

In [10]:
model = model_definition.UNet(input_size = (512, 512, 1))
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_24 (Conv2D)             (None, 512, 512, 64  640         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_25 (Conv2D)             (None, 512, 512, 64  36928       ['conv2d_24[0][0]']              
                                )                                                           

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Train the model

In [69]:
a = model_train.train_model(model, X_train, X_test, y_train, y_test, IMG_LIST_DIR, SEG_LIST_DIR)

(1, 512, 512, 1)
(1, 512, 512, 1)
(2, 512, 512, 1)
(2, 512, 512, 1)
Epoch 1/3


TypeError: ignored